In [9]:
import strax
import pandas as pd
pd.options.display.max_colwidth = 100
from tqdm import tqdm
import numpy as np
import numba
from scipy.optimize import curve_fit
from scipy import stats
import straxen
st = straxen.contexts.strax_workshop_dali()

In [10]:
#@export
@strax.takes_config(
    strax.Option(
        'LED_window',
        default=(125, 250),
        help="Window (samples) where we expect signal in LED calibration"))


def get_amplitude(raw_records, windows):  
    on = []
    off = []
    for r in raw_records:
        amp_LED = np.max(r['data'][window[0]:window[1]])
        amp_NOISE = np.max(r['data'][2*window[0]:2*window[1]])
        on.append(amp_LED)
        off.append(amp_NOISE)
    on = np.array(on, dtype=[('amplitudeLED', '<i4')])
    off = np.array(off, dtype=[('amplitudeNOISE', '<i4')])
    return on, off

def get_area(raw_records, windows):
    left = window[0]
    end_pos = [window[1]+2*i for i in range(6)]
    n_channel_s = np.arange(0, 249, 1)
    Area = np.array(raw_records[['channel', 'area']],dtype=[('channel','int16'),('area','float32')])

    for n_channel in n_channel_s:
        wf_tmp = raw_records[raw_records['channel'] == n_channel]
        area = 0
        for right in end_pos:
            area += wf_tmp['data'][:,left:right].sum(axis=1)

        mask = np.where(Area['channel'] == n_channel)[0]
        Area['area'][mask] = area.astype(np.float)/6.

    return Area

    
class LEDCalibration(strax.Plugin):
    '''
    Let's start with something really easy.
    LEDCalibration gives -> look dtype
    '''   
    __version__ = '0.0.4'

    depends_on = ('raw_records',)
    data_kind = 'led_cal_0'
    compressor = 'zstd'
    parallel = 'process'
    rechunk_on_save = False
    dtype = [('area', np.int32, 'Area'),
             ('amplitudeLED', np.int32, 'Amplitude in LED window'),
             ('amplitudeNOISE', np.int32, 'Amplitude in off LED window'),
             ('channel', np.int16, 'Channel'),
             ('time', np.int64, 'Start time of the interval (ns since unix epoch)'),
             ('dt', np.int16, 'Time resolution in ns'),
             ('length', np.int32, 'Length of the interval in samples')
            ]

    def compute(self, raw_records):
        # Remove records from funny channels (if present)
        r = raw_records[raw_records['channel'] < 248] # hardcoded for now
        temp = np.zeros(len(r), dtype=self.dtype)
        
        temp['channel'] = r['channel']
        temp['time'] = r['time']
        temp['dt'] = r['dt']
        temp['length'] = r['length']
        
        #info, window = get_LED_window(r)
        #print(window)
        on, off = get_amplitude(r, self.config['LED_window'])
        temp['amplitudeLED'] = on
        temp['amplitudeNOISE'] = off
        area = get_area(r, self.config['LED_window'])
        print(area['channel'] == r['channel'])
        temp['area'] = area['area']
        
        return temp

In [11]:
st = straxen.contexts.strax_workshop_dali()
st.register(LEDCalibration)
st.data_info('led_calibration')

,Field name,Data type,Comment
0,area,int32,Area
1,amplitudeLED,int32,Amplitude in LED window
2,amplitudeNOISE,int32,Amplitude in off LED window
3,channel,int16,Channel
4,time,int64,Start time of the interval (ns since unix epoch)
5,dt,int16,Time resolution in ns
6,length,int32,Length of the interval in samples


In [12]:
st.show_config('led_calibration', 'LED*')

""
